In [ ]:
#                                 #          In the Name of GOD   # #
#
import numpy as np
import scipy.sparse as sp
from concurrent.futures import ProcessPoolExecutor

class MultilayerNetwork:
    
    def __init__(self, directed : bool = False, large_graph : bool = False):
        
        self.directed = directed
        self.large_graph = large_graph
        self.node_set = set()
        self.nodes = {}#                   Nodes by layer
        self.edges = {}#                   Adjacency matrices or arrays by layer
        self.layers = []#                  List of layer names
        self.node_attributes = {}
        self.edge_attributes = {}
        self.inter_layer_edges = []#       Managing inter-layer edges
    
    
    def add_layer(self, layer_name : str ):
        
        if layer_name not in self.layers:
            
            self.layers.append(layer_name)
            self.nodes[layer_name] = []
            self.edges[layer_name] = []
    
    
    def add_node(self, layer_name : str , node ):
        
        if layer_name not in self.layers:
            self.add_layer(layer_name)
        
        if node not in self.nodes[layer_name]:
            self.nodes[layer_name].append(node)
            self.node_set.add(node)
            
            if self.large_graph:
                if layer_name not in self.edges or not isinstance(self.edges[layer_name], sp.lil_matrix):
                    self.edges[layer_name] = sp.lil_matrix((1, 1))
            else:
                if layer_name not in self.edges or not isinstance(self.edges[layer_name], np.ndarray):
                    self.edges[layer_name] = np.zeros((1, 1), dtype=int)
                
                old_matrix = self.edges[layer_name]
                new_size = len(self.nodes[layer_name])
                new_matrix = np.zeros((new_size, new_size), dtype=int)
                new_matrix[:old_matrix.shape[0], :old_matrix.shape[1]] = old_matrix
                self.edges[layer_name] = new_matrix
    
    
    def add_edge(self, node1, node2, layer_name1 : str, layer_name2 : str = None, weight : int | float = 1 ):
        
        if layer_name2 is None or layer_name1 == layer_name2:
            
            if layer_name1 not in self.layers:
                raise ValueError(f"Layer '{layer_name1}' does not exist.")
            
            if node1 not in self.nodes[layer_name1] or node2 not in self.nodes[layer_name1]:
                raise ValueError("One or both nodes do not exist in the specified layer.")
            
            self._ensure_correct_matrix_size(layer_name1, node1, node2)
            node1_index = self.nodes[layer_name1].index(node1)
            node2_index = self.nodes[layer_name1].index(node2)
            
            self.edges[layer_name1][node1_index, node2_index] = weight
            
            if not self.directed:
                self.edges[layer_name1][node2_index, node1_index] = weight
        
        else:
            self.add_inter_layer_edge(node1, node2, layer_name1, layer_name2, weight)
    
    
    def add_inter_layer_edge(self, node1, node2, layer_name1 : str , layer_name2 : str , weight : int | float = 1 ):
        
        if layer_name1 not in self.layers or layer_name2 not in self.layers:
            raise ValueError(f"One or both specified layers ('{layer_name1}', '{layer_name2}') do not exist.")
        
        if node1 not in self.nodes.get(layer_name1, []) or node2 not in self.nodes.get(layer_name2, []):
            raise ValueError("One or both nodes do not exist in their specified layers.")
        
        self.inter_layer_edges.append(((node1, layer_name1), (node2, layer_name2), weight))
    
    
    def _ensure_correct_matrix_size(self, layer_name : str , node1, node2):
        
        node1_index = self.nodes[layer_name].index(node1)
        node2_index = self.nodes[layer_name].index(node2)
        
        if self.large_graph:
            self._ensure_matrix_initialized_and_resized(layer_name, node1_index, node2_index)
        else:
            self._ensure_dense_matrix_resized(layer_name, node1_index, node2_index)
    
    
    def _ensure_matrix_initialized_and_resized(self, layer_name : str , node1_index, node2_index):
        
        if layer_name not in self.edges or self.edges[layer_name].shape == (0, 0):
            self.edges[layer_name] = sp.lil_matrix((1, 1))
        
        current_matrix = self.edges[layer_name]
        max_index = max(node1_index, node2_index) + 1
        
        if max_index > current_matrix.shape[0]:
            new_matrix = sp.lil_matrix((max_index, max_index))
            new_matrix[:current_matrix.shape[0], :current_matrix.shape[1]] = current_matrix
            self.edges[layer_name] = new_matrix
    
    
    def _ensure_dense_matrix_resized(self, layer_name : str, node1_index, node2_index):
        
        if layer_name not in self.edges or not len(self.edges[layer_name]):
            self.edges[layer_name] = np.zeros((1, 1), dtype=int)
        
        current_matrix = np.array(self.edges[layer_name])
        max_index = max(node1_index, node2_index) + 1
        
        if max_index > current_matrix.shape[0]:
            new_matrix = np.zeros((max_index, max_index), dtype=int)
            new_matrix[:current_matrix.shape[0], :current_matrix.shape[1]] = current_matrix
            self.edges[layer_name] = new_matrix.tolist() if isinstance(self.edges[layer_name], list) else new_matrix
    
    
    def get_inter_layer_edges(self):
        
        return self.inter_layer_edges
    
    
    def find_inter_layer_edges(self, node, layer : str = None):
        """
        Find all inter-layer edges connected to a given node, optionally within a specific layer.
        """
        if layer:
            return [(n1, l1, n2, l2, w) for (n1, l1, n2, l2, w) in self.inter_layer_edges if (node == n1 and layer == l1) or (node == n2 and layer == l2)]
        else:
            return [(n1, l1, n2, l2, w) for (n1, l1, n2, l2, w) in self.inter_layer_edges if node == n1 or node == n2]
    
    
    def prepare_for_bulk_update(self, layer_name : str ):
        """
        Prepare or reset the accumulator for a layer.
        """
        self.bulk_updates[layer_name] = {'rows': [], 'cols': [], 'data': []}
    
    
    def accumulate_edge_update(self, node1, node2, layer_name : str, weight=1):
        """
        Accumulate an edge update for later bulk application.
        """
        if layer_name not in self.nodes:
            raise ValueError("Layer does not exist.")
        if node1 not in self.nodes[layer_name] or node2 not in self.nodes[layer_name]:
            raise ValueError("One or both nodes do not exist in the specified layer.")
        
        node1_index = self.nodes[layer_name].index(node1)
        node2_index = self.nodes[layer_name].index(node2)
        
        self.bulk_updates[layer_name]['rows'].append(node1_index)
        self.bulk_updates[layer_name]['cols'].append(node2_index)
        self.bulk_updates[layer_name]['data'].append(weight)
    
    
    def update_edge_weight(self, node1, node2, layer_name : str, new_weight : int | float ):
        
        if layer_name not in self.layers or node1 not in self.nodes[layer_name] or node2 not in self.nodes[layer_name]:
            raise ValueError("Layer, node1, or node2 does not exist.")
        
        node1_index = self.nodes[layer_name].index(node1)
        node2_index = self.nodes[layer_name].index(node2)
        self.edges[layer_name][node1_index, node2_index] = new_weight
        
        if not self.directed:
            self.edges[layer_name][node2_index, node1_index] = new_weight
    
    
    def apply_bulk_updates(self, layer_name : str ):
        """ 
        Apply accumulated updates in bulk to the adjacency matrix of a layer. 
        """
        if layer_name not in self.bulk_updates:
            return  # No updates to apply
        
        update_data = self.bulk_updates[layer_name]
        if self.large_graph:
            # for large (sparse) graphs
            update_matrix = sp.coo_matrix((update_data['data'], (update_data['rows'], update_data['cols'])),
                                        shape=self.edges[layer_name].shape)
            self.edges[layer_name] += update_matrix.tocsr()
        else:
        # for small (dense) graphs
            for row, col, data in zip(update_data['rows'], update_data['cols'], update_data['data']):
                self.edges[layer_name][row, col] = data
                if not self.directed:
                    self.edges[layer_name][col, row] = data
        
        # reset the accumulator for the layer
        self.prepare_for_bulk_update(layer_name)
    
    
    def set_node_attribute(self, node, attr_name : str , attr_value : int | float | list ):
        """
        Set or update an attribute for a node.
        Args:
        node (str or int): The node identifier.
        attr_name (str): The name of the attribute.
        attr_value (any): The value of the attribute.
        """
        if node not in self.node_set:
            raise ValueError("Node does not exist.")
        if node not in self.node_attributes:
            self.node_attributes[node] = {}
        self.node_attributes[node][attr_name] = attr_value
    
    
    def set_edge_attribute(self, node1, node2, layer_name : str, attr_name : str , attr_value : int | float ):
        if node1 not in self.node_set or node2 not in self.node_set:
            raise ValueError("One or both nodes do not exist.")
        if layer_name not in self.layers:
            raise ValueError(f"Layer {layer_name} does not exist.")
        
        edge_key = (node1, node2, layer_name) if self.directed else (min(node1, node2), max(node1, node2), layer_name)
        
        self.edge_attributes[edge_key] = self.edge_attributes.get(edge_key, {})
        self.edge_attributes[edge_key][attr_name] = attr_value
    
    
    def calculate_layer_degrees(self, layer_name : str , parallel_threshold=10000):
        
        if layer_name not in self.layers:
            raise ValueError(f"Layer {layer_name} not found.")
        layer_matrix = self.edges[layer_name]
        
        node_count = layer_matrix.shape[0] if self.large_graph else len(layer_matrix)
        
        if self.large_graph and node_count > parallel_threshold:
            with ProcessPoolExecutor() as executor:
                func_args = [(self.large_graph, layer_matrix, i) for i in range(node_count)]
                degrees = list(executor.map(MultilayerNetwork._calculate_degree_of_node, func_args))
        else:
            degrees = self._calculate_layer_degrees_single_threaded(layer_name)
        
        return degrees
    
    
    def _calculate_layer_degrees_single_threaded(self, layer_name : str ):
        
        layer_matrix = self.edges[layer_name]
        
        if self.large_graph:
            
            if self.directed:
                in_degrees = layer_matrix.sum(axis=0).A1  # Column sum for in-degree
                out_degrees = layer_matrix.sum(axis=1).A1  # Row sum for out-degree
                return in_degrees, out_degrees
            
            else:
                degrees = layer_matrix.sum(axis=1).A1  # Row sum suffices
                return degrees
        else:
            degrees = np.sum(layer_matrix > 0, axis=1)  # For dense matrices
            return degrees
    
    
    @staticmethod
    def _calculate_degree_of_node(args):
        
        large_graph, layer_matrix, node_index = args
        
        if large_graph:
            # In this case layer_matrix is a sparse matrix
            row = layer_matrix.getrow(node_index)
            return row.getnnz()
        
        else:
            # In this case layer_matrix is a dense numpy array
            return np.sum(layer_matrix[node_index] > 0)

#end#

In [ ]:
#                                 #          In the Name of GOD   # #
#
import numpy as np
import scipy.sparse as sp
from scipy.sparse.csgraph import connected_components, shortest_path, dijkstra
from scipy.sparse.linalg import eigs
from sklearn.cluster import SpectralClustering
from joblib import Parallel, delayed


class MNAnalysis:
    
    def __init__(self, multilayer_network):
        """
        Initialize the analysis class with a MultilayerNetwork instance.
        """
        self.network = multilayer_network
    
    
    def layerwise_degree_distribution(self):
        """
        Calculate the degree distribution for each layer of the network.
        """
        degree_distributions = {}
        for layer in self.network.layers:
            degrees = self.network.calculate_layer_degrees(layer)
            degree_distributions[layer] = np.bincount(degrees) / float(len(degrees))
        return degree_distributions
    
    
    def aggregate_network(self):
        """
        Aggregate the multilayer network into a single-layer network.
        This method combines all layers into one, summing up the weights of inter-layer edges.
        """
        aggregated_matrix = None
        for layer in self.network.layers:
            matrix = self.network.edges[layer]
            if isinstance(matrix, sp.lil_matrix):
                matrix = matrix.tocsr()
            if aggregated_matrix is None:
                aggregated_matrix = matrix
            else:
                aggregated_matrix += matrix
        return aggregated_matrix
    
    
    def detect_communities(self, layer_name, n_clusters=2):
        """
        Detect communities within a specific layer using spectral clustering.
        """
        if layer_name not in self.network.layers:
            raise ValueError(f"Layer {layer_name} not found. Ensure the layer name is correct.")
        
        adjacency_matrix = self.network.edges[layer_name]
        if not sp.issparse(adjacency_matrix):
            adjacency_matrix = sp.csr_matrix(adjacency_matrix)
        
        clustering = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
        labels = clustering.fit_predict(adjacency_matrix)
        
        return labels
    
    
    def calculate_global_efficiency(self, layer_name):
        matrix = self.network.edges[layer_name]
        if not sp.issparse(matrix):
            matrix = sp.csr_matrix(matrix)
        
        distances = dijkstra(matrix, directed=self.network.directed, unweighted=True)
        finite_distances = distances[np.isfinite(distances) & (distances > 0)]
        
        if finite_distances.size == 0:
            return 0  # Return 0 efficiency if there are no valid paths
        
        efficiency = np.mean(1. / finite_distances)
        return efficiency
    
    
    def count_connected_components(self, layer_name):
        """
        Count the number of connected components in a specific layer.
        """
        if layer_name not in self.network.layers:
            raise ValueError(f"Layer {layer_name} not found.")
        
        matrix = self.network.edges[layer_name]
        if isinstance(matrix, list):
            matrix = np.array(matrix)
        if not sp.issparse(matrix):
            matrix = sp.csr_matrix(matrix)
        
        n_components, _ = connected_components(csgraph=matrix, directed=self.network.directed, return_labels=True)
        return n_components
    
    
    def analyze_dynamic_changes(self, snapshots):
        """
        Analyze dynamic changes in the network over a series of snapshots. Each snapshot is a MultilayerNetwork instance.
        Returns a list of changes in global efficiency over time.
        """
        efficiencies = []
        for snapshot in snapshots:
            efficiency_per_layer = {}
            for layer_name in snapshot.layers:
                efficiency = self.calculate_global_efficiency(layer_name)
                efficiency_per_layer[layer_name] = efficiency
            efficiencies.append(efficiency_per_layer)
        return efficiencies
    
    
    def explore_inter_layer_connectivity(self):
        """
        Explore and quantify the connectivity patterns between layers.
        This method calculates the density of inter-layer edges and the distribution of weights.
        """
        inter_layer_edges = self.network.get_inter_layer_edges()
        total_inter_layer_edges = len(inter_layer_edges)
        if total_inter_layer_edges == 0:
            return {'density': 0, 'weight_distribution': []}
        
        total_possible_inter_layer_edges = sum(len(self.network.nodes[layer]) for layer in self.network.layers) ** 2
        density = total_inter_layer_edges / total_possible_inter_layer_edges
        
        weights = [weight for _, _, _, _, weight in inter_layer_edges]
        weight_distribution = np.histogram(weights, bins=10, density=True)[0]
        
        return {'density': density, 'weight_distribution': weight_distribution.tolist()}
    
    
    def parallel_betweenness_centrality(self, layer_name):
        matrix = self.network.edges[layer_name]
        if not isinstance(matrix, sp.csr_matrix):
            matrix = sp.csr_matrix(matrix)
        n = matrix.shape[0]
        
        def compute_for_node(start):
            _, predecessors = shortest_path(csgraph=matrix, directed=self.network.directed, indices=start, return_predecessors=True)
            betweenness = np.zeros(n)
            
            for end in range(n):
                if start == end:
                    continue
                path = []
                intermediate = end
                while intermediate != start:
                    path.append(intermediate)
                    intermediate = predecessors[intermediate]
                    if intermediate == -9999:  # Check for unreachable nodes
                        path = []
                        break
                path.reverse()
                for node in path[1:-1]:
                    betweenness[node] += 1
            
            return betweenness
        
        results = Parallel(n_jobs=-1)(delayed(compute_for_node)(i) for i in range(n))
        total_betweenness = np.sum(results, axis=0)
        total_betweenness /= 2  # to account for each path being counted twice in an undirected graph
        return total_betweenness.tolist()
    
    
    def calculate_centrality_measures(self, layer_name, use_weight=False):
        
        if layer_name not in self.network.layers:
            raise ValueError(f"Layer {layer_name} not found.")
        
        matrix = self.network.edges[layer_name]
        
        if isinstance(matrix, list):
            matrix = np.array(matrix)
        if not sp.issparse(matrix):
            matrix = sp.csr_matrix(matrix)
        
        # Degree Centrality
        if sp.issparse(matrix):
            if use_weight:
                degree_centrality = matrix.sum(axis=0).A1 / (matrix.shape[0] - 1)
            else:
                degree_centrality = (matrix != 0).sum(axis=0).A1 / (matrix.shape[0] - 1)
        else:
            if use_weight:
                degree_centrality = matrix.sum(axis=1) / (matrix.shape[0] - 1)
            else:
                degree_centrality = (matrix != 0).sum(axis=1) / (matrix.shape[0] - 1)
        
        # Betweenness Centrality
        betweenness_centrality = self._calculate_betweenness_centrality(matrix, matrix.shape[0], use_weight)
        
        # Eigenvector Centrality
        eigenvector_centrality = self._calculate_eigenvector_centrality(matrix)
        
        centralities = {
            'degree_centrality': degree_centrality.tolist(),
            'betweenness_centrality': betweenness_centrality,
            'eigenvector_centrality': eigenvector_centrality
        }
        
        return centralities
    
    
    def calculate_centrality_with_attributes(self, layer_name, attribute_name, use_weight=False):
        
        if layer_name not in self.network.layers:
            raise ValueError(f"Layer {layer_name} not found.")
        
        matrix = self.network.edges[layer_name]
        if isinstance(matrix, list):
            matrix = np.array(matrix)
        if not sp.issparse(matrix):
            matrix = sp.csr_matrix(matrix)
        
        # Retrieve node attributes
        attributes = [self.network.node_attributes.get(node, {}).get(attribute_name, 1) for node in self.network.nodes[layer_name]]
        
        # Adjust matrix for attributes if using weights
        if use_weight:
            attr_matrix = sp.diags(attributes)
            matrix = attr_matrix @ matrix if sp.issparse(matrix) else np.diag(attributes) @ matrix
        
        # Degree Centrality with attributes
        degree_centrality = matrix.sum(axis=0).A1 / (matrix.shape[0] - 1) if sp.issparse(matrix) else matrix.sum(axis=1) / (matrix.shape[0] - 1)
        
        # Betweenness Centrality with attributes
        betweenness_centrality = self._calculate_betweenness_centrality(matrix, matrix.shape[0], use_weight)
        
        # Eigenvector Centrality with attributes
        eigenvector_centrality = self._calculate_eigenvector_centrality(matrix)
        
        centralities = {
            'degree_centrality': degree_centrality.tolist(),
            'betweenness_centrality': betweenness_centrality,
            'eigenvector_centrality': eigenvector_centrality
        }
        
        return centralities
    
    
    def _calculate_betweenness_centrality(self, matrix, n, use_weight=False):
        """
        Calculate the betweenness centrality for each node in a weighted or unweighted graph.
        """
        if use_weight:
            # Use Dijkstra's algorithm for weighted graphs
            dist_matrix, predecessors = shortest_path(csgraph=matrix, directed=self.network.directed, return_predecessors=True, unweighted=False)
        else:
            # Unweighted graph, use faster Floyd-Warshall algorithm
            dist_matrix, predecessors = shortest_path(csgraph=matrix, directed=self.network.directed, return_predecessors=True, unweighted=True)
        
        betweenness = np.zeros(n)
        for source in range(n):
            for target in range(n):
                if source != target:
                    # Reconstruct the shortest path from source to target
                    path = []
                    intermediate = target
                    while predecessors[source, intermediate] != -9999:
                        path.append(intermediate)
                        intermediate = predecessors[source, intermediate]
                        if intermediate == source:
                            break
                    path.append(source)
                    path.reverse()

                    # Count the betweenness
                    for v in path[1:-1]:  # exclude the source and target themselves
                        betweenness[v] += 1
        
        # Normalize the betweenness scores
        if not self.network.directed:
            betweenness /= 2
        return betweenness.tolist()
    
    
    def _calculate_eigenvector_centrality(self, matrix, use_weight=False):
        """
        Calculate eigenvector centrality using the power iteration method.
        Handles both sparse and dense matrix formats. Provides an option to consider or ignore edge weights.
        
        Args:
        matrix (np.ndarray or sp.spmatrix): The adjacency matrix of the network.
        use_weight (bool): If True, use the edge weights as given; if False, treat the graph as unweighted.
        """
        try:
            if not use_weight:
                # Convert all non-zero entries to 1 to ignore actual weights
                if sp.issparse(matrix):
                    matrix = sp.csr_matrix((np.ones_like(matrix.data), matrix.indices, matrix.indptr), shape=matrix.shape)
                else:
                    matrix = np.where(matrix != 0, 1, 0)
            
            if sp.issparse(matrix):
                matrix = matrix.astype(np.float64)  # Ensure matrix is of floating-point type
                eigenvalue, eigenvector = eigs(A=matrix, k=1, which='LR', maxiter=10000, tol=1e-6)
            else:
                matrix = np.array(matrix, dtype=np.float64)  # Ensure matrix is of floating-point type
                eigenvalue, eigenvector = np.linalg.eig(matrix)
                largest = np.argmax(eigenvalue)
                eigenvector = eigenvector[:, largest]
            
            eigenvector_centrality = np.abs(np.real(eigenvector)) / np.linalg.norm(np.real(eigenvector), 1)
            return eigenvector_centrality.tolist()
        
        except Exception as e:
            error_message = f"Failed to calculate eigenvector centrality. Ensure the matrix is appropriate for this operation. Error: {e}"
            raise RuntimeError(error_message)

#end#

**install it if you did not yet!**

**!pip install simple-network**

**import library :**

from simpleN import MultilayerNetwork, Visualize

In [2]:
# Create an Object of it :

g = MultilayerNetwork()

In [3]:
# adding layers !

g.add_layer('one')
g.add_layer('two')

In [4]:
# adding nodes to layer one !

for i in range(50):
    g.add_node(layer_name='one', node = i)

In [5]:
# adding nodes to layer two !

for i in range(50, 100):
    g.add_node(layer_name='two', node = i)

In [6]:
# Add Between layer edges !

g.add_edge(node1=5, node2= 80 , layer_name1= 'one', layer_name2='two')

g.add_edge(node1=17, node2= 55 , layer_name1= 'one', layer_name2='two')

g.add_edge(node1=6, node2= 90 , layer_name1= 'one', layer_name2='two')

g.add_edge(node1=47, node2= 52 , layer_name1= 'one', layer_name2='two')

g.add_edge(node1=5, node2= 60 , layer_name1= 'one', layer_name2='two')

g.add_edge(node1=15, node2= 99 , layer_name1= 'one', layer_name2='two')

In [7]:
# Add Inside layer edges !

#g.add_edge(node1=6, node2= 27 , layer_name1= 'one')

g.add_edge(node1=22, node2= 47 , layer_name1= 'one')

g.add_edge(node1=14, node2= 15 , layer_name1= 'one')

g.add_edge(node1=71, node2= 90 , layer_name1= 'two')

g.add_edge(node1=66, node2= 88 , layer_name1= 'two')

In [ ]:
g.set_node_attribute(node=1, attr_name= 'RG', attr_value= 1.15)




g.set_node_attribute(node=6, attr_name= 'RG', attr_value= 11)


g.set_node_attribute(node=7, attr_name= 'RG', attr_value= 1.2)

g.set_node_attribute(node=5, attr_name= 'RG', attr_value= 1.15)
g.set_node_attribute(node=70, attr_name= 'RG', attr_value=1.5)
g.set_node_attribute(node=80, attr_name= 'RG', attr_value=2.2)
g.set_node_attribute(node=47, attr_name= 'RG', attr_value= 2)
g.set_node_attribute(node=52, attr_name= 'RG', attr_value=1.5)

g.set_node_attribute(node=11, attr_name= 'PP', attr_value= 1.15)


g.set_node_attribute(node=90, attr_name= 'PP', attr_value= 4.9)

g.set_node_attribute(node=88, attr_name= 'RG', attr_value= 22)
g.set_node_attribute(node=27, attr_name= 'RG', attr_value= 21.22)
g.set_node_attribute(node=87, attr_name= 'PP', attr_value= 1.2)
g.set_node_attribute(node=66, attr_name= 'RG', attr_value= 22)
g.set_node_attribute(node=86, attr_name= 'PP', attr_value=1.5)

In [8]:
# adj_matrix = g.edges

In [12]:
Visualize(g).show_graph()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

class MultilayerNetworkDataset(Dataset):
    """
    A PyTorch Dataset class to handle node features and labels for the MultilayerNetwork.
    """
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)  # Features should already be processed
        self.labels = torch.tensor(labels, dtype=torch.float32)  # Assuming labels are already processed

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class LearnableNetwork:
    def __init__(self, multilayer_network, model=None):
        """
        Initialize LearnableNetwork with a reference to an existing multilayer network instance.
        Allows for a custom model to be passed or uses a simple default model.
        """
        self.network = multilayer_network
        if model is None:
            # Use a default simple model if none provided
            feature_size = len(next(iter(multilayer_network.node_attributes.values())))
            output_size = len(multilayer_network.node_set)
            self.model = nn.Sequential(
                nn.Linear(feature_size, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, output_size),
                nn.Sigmoid()
            )
        else:
            self.model = model

    def extract_features(self):
        """
        Feature extraction from multilayer network for learning.
        Returns features and labels suitable for training.
        """
        features = []
        labels = []
        for node in self.network.node_set:
            node_features = list(self.network.node_attributes[node].values())
            node_labels = [1 if other in self.network.edges[self.network.layers[0]][node] else 0
                           for other in self.network.node_set]
            features.append(node_features)
            labels.append(node_labels)

        return features, labels

    def train(self, features, labels, epochs=10, batch_size=32, lr=0.01):
        """
        Train the model using the provided features and labels.
        """
        dataset = MultilayerNetworkDataset(features, labels)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        criterion = nn.BCELoss()

        self.model.train()
        for epoch in range(epochs):
            for features, labels in dataloader:
                optimizer.zero_grad()
                outputs = self.model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    def predict(self, features):
        """
        Make predictions based on the node features.
        """
        self.model.eval()
        with torch.no_grad():
            features = torch.tensor(features, dtype=torch.float32)
            predictions = self.model(features)
        return predictions.numpy()

    def update_network(self, predictions, threshold=0.5):
        """
        Apply learned insights or predictions to update the multilayer network.
        """
        for idx, node in enumerate(self.network.node_set):
            for jdx, predicted in enumerate(predictions[idx]):
                if predicted > threshold and idx != jdx:  # Avoid self-loops and weak connections
                    self.network.add_edge(node, list(self.network.node_set)[jdx], self.network.layers[0])


